# Functions with Multiple Parameters vs. Curried Functions  

In [71]:
// Multiple argument functions
def addTwo (x: Int, y: Int): Int = x + y

val x = addTwo(10) // Error

cmd71.sc:3: not enough arguments for method addTwo: (x: Int, y: Int)Int.
Unspecified value parameter y.
val x = addTwo(10) // Error
              ^Compilation Failed

: 

Note the type of addTwo is `(Int, Int) => Int`

In [8]:
// Scala supports curried functions: sort of.

def curriedAddTwo (x: Int) (y: Int) = x + y 

defined function curriedAddTwo

Note the type of curriedAddTwo is `Int => (Int => Int)`.

  - If you give it a number, it returns a function from number to number.

In [ ]:
val f1 = curriedAddTwo(15)
// Oops.. scala does not allow this syntax although lettuce does.

In [ ]:
val f1 = curriedAddTwo (15)(_)
// f1 has plugged in x but y has not be set. Remember _ makes it an anonymous function
val x = f1(20)
val y = f1(30)
val z = f1(45)

In [ ]:
// Currying in anonymous function syntax
val curriedAddTwoAnon = (x: Int) => { (y:Int) => (x + y) } 

In [ ]:
val f1 = curriedAddTwoAnon(15) // Now this is closer to lettuce syntax
val f2 = curriedAddTwoAnon(15)(_) // This is also OK

# Handling Recursion using Environments

Below is a Scala implementation of the circular scope

In [17]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr {
    override def toString(): String = f.toString
}
case class Ident(s: String) extends Expr {
    override def toString(): String = s
}
case class Minus(e1: Expr, e2: Expr) extends Expr {
    override def toString(): String = s"(${e1.toString} - ${e2.toString})"
}
case class Plus(e1: Expr, e2: Expr) extends Expr {
    override def toString(): String = s"(${e1.toString} + ${e2.toString})"
}
case class Mult(e1: Expr, e2: Expr) extends Expr {
    override def toString(): String = s"(${e1.toString} * ${e2.toString})"
}
case class Eq(e1: Expr, e2: Expr) extends Expr {
    override def toString(): String = s"(${e1.toString} == ${e2.toString})"
}
case class Geq(e1: Expr, e2: Expr) extends Expr {
    override def toString(): String = s"(${e1.toString} >= ${e2.toString})"
}
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr {
    override def toString(): String = s"if (${e1.toString}) ${e2.toString} else ${e3.toString}"
}
case class Let(x: String, e1: Expr, e2: Expr) extends Expr {
    override def toString(): String = s"let $x = ${e1.toString} in ${e2.toString}"
}
case class FunDef(id: String, e: Expr) extends Expr {
    override def toString(): String = s"fun $id = ${e.toString}"
}
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr {
    override def toString(): String = s"$calledFun(${argExpr.toString})"
}
/* Here is the new stuff */
case class LetRec(funName: String, param: String, funExpr: Expr, bodyExpr: Expr) extends Expr {
    override def toString(): String = s"let rec $funName($param) = ${funExpr.toString} in ${bodyExpr.toString}"
}

case class TopLevel(e: Expr) extends Program {
    override def toString(): String = e.toString
}

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Minus
defined class Plus
defined class Mult
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class TopLevel

In [62]:
sealed trait Environment 
sealed trait Value

case object EmptyEnv extends Environment {
    override def toString(): String = s"()"
}
case class Extend(x: String, v: Value, sigma: Environment) extends Environment {
    override def toString(): String = s"$x -> ${v.toString}, $sigma"
}
//ExtendRec helps me create the illusion of a circular scope
case class ExtendRec(f: String, x: String, e: Expr, sigma: Environment ) extends Environment {
    override def toString(): String = s"$f -> Closure($x, ${e.toString}, ($f -> THIS_ENV, $sigma))"
}

/* -- We need to redefine values to accomodate the new representation of environments --*/
case class NumValue(d: Double) extends Value {
    override def toString(): String = d.toString
}
case class BoolValue(b: Boolean) extends Value {
    override def toString(): String = b.toString
}
case class Closure(x: String, e: Expr, pi: Environment) extends Value {
    override def toString(): String = s"Closure($x, ${e.toString}, ${pi.toString})"
}
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


/*-- Operations on environments --*/

def lookupEnv(sigma: Environment, x: String): Value = {
    val ret = {
            sigma match {
            case EmptyEnv => throw new IllegalArgumentException(s"Error could not find string $x in environment")
            case Extend(y, v, rest) =>
                if (y == x ) { v }
                else { lookupEnv (rest, x) }
            // The illusion of circularity is here.
            case ExtendRec(funName, param, funBody, rest) =>
                if (x == funName) 
                    Closure(param, funBody, sigma)
                else
                    lookupEnv(rest, x)
        }
    }
    println(s"Look up variable `$x` in environment `$sigma`. Value is `$ret`.")
    ret
}

defined trait Environment
defined trait Value
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined function lookupEnv

In [67]:
/*-- We can rewrite evalExpr now to handle recursion as well --*/
def evalExpr(e: Expr, env: Environment): Value =  {
    println(s"Evaluate expression `${e.toString}` in environment `$env`")
    
    /* Method to deal with binary arithmetic operations */
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f) // Same as before
        
        case Ident(x) => lookupEnv(env, x) // Changed to accomodate the new environment definitions.
    
        /* Ditto as before */
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
        /* Ditto as before */
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
        /* Ditto as before */
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        /* Ditto as before */
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        /* Ditto as before */
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        /* Ditto as before */
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        /* Ditto as before */
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = Extend(x, v1, env) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
        /* Ditto as before */
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        /* Ditto as before */
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = Extend(x, v2, closed_env)
                    // Evaluate the body of the closure under the extended environment.
                    println(s"Evaluate function body `$closure_ex` in environment `$new_env`")
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        /*-- This is where we handle recursion --*/
        case LetRec(rfun, x, fExpr, bExpr) => {
            // Storing all the ingredients that are needed to create
            // a circular scope.
            val env2 = ExtendRec(rfun, x, fExpr, env)
            evalExpr(bExpr, env2)
        }
    }
}

def evalProgram(p: Program) = {
    p match { 
        case TopLevel(e) => evalExpr(e, EmptyEnv)
    }
}

defined function evalExpr
defined function evalProgram

In [70]:
val fact_rec_1 = TopLevel(
    LetRec("fact", "n", 
          IfThenElse(Eq(Ident("n"), Const(0)), Const(1), 
                     Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1))))
                    ),
           FunCall(Ident("fact"), Const(1))
          )
)
evalProgram(fact_rec_1)

Evaluate expression `let rec fact(n) = if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))) in fact(1.0)` in environment `()`
Evaluate expression `fact(1.0)` in environment `fact -> Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), (fact -> THIS_ENV, ()))`
Evaluate expression `fact` in environment `fact -> Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), (fact -> THIS_ENV, ()))`
Look up variable `fact` in environment `fact -> Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), (fact -> THIS_ENV, ()))`. Value is `Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), fact -> Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), (fact -> THIS_ENV, ())))`.
Evaluate expression `1.0` in environment `fact -> Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), (fact -> THIS_ENV, ()))`
Evaluate function body `if ((n == 0.0)) 1.0 else (n * fact((n - 1.0)))` in environment `n -> 1.0, fact -> Closure(n, if ((n == 0.0)) 1.0 else (n * fact((n - 1.0))), (fact -> T

fact_rec_1: TopLevel = TopLevel(
  LetRec(
    "fact",
    "n",
    IfThenElse(
      Eq(Ident("n"), Const(0.0)),
      Const(1.0),
      Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1.0))))
    ),
    FunCall(Ident("fact"), Const(1.0))
  )
)
res69_1: Value = NumValue(1.0)

## Illustration of circular scoping for function `fact`

![Circular scope](circular_scope.png)